# Weather parameters forcast

Firstly we import all necessary librairies

In [4]:
!pip install xgboost

In [2]:
import pandas as pd
import numpy as np
import requests
import os
import datetime
import time
import joblib

In [3]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.ensemble import RandomForestRegressor
from xgboost import XGBRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score


In this cell we define some fixed parameters to be used in the code, they can be modified dynamically if needed

In [4]:
API_URL = "https://api.open-meteo.com/v1/forecast"
LAT, LON = 33.8886, 35.4950 # for beirut
PARAMS = {"latitude": LAT, "longitude": LON, "hourly": ["temperature_2m", "pressure_msl", "relative_humidity_2m", "cloudcover", "windspeed_10m", "winddirection_10m"], "past_days": 7, "timezone": "UTC"}
FILE_PATH = "weather_data.xlsx"
MAX_WEEKS = 270 # firstly from 1-1-2020 t0 1-3-2025

This function is for season feature, it help the fetched data to decide for which season it can be part

In [5]:
def get_season(month):
    seasons = {
        (12, 1, 2): 'Winter',
        (3, 4, 5): 'Spring',
        (6, 7, 8): 'Summer',
        (9, 10, 11): 'Fall'
    }
    for months, season in seasons.items():
        if month in months:
            return season
    return 'Unknown'


This function help in fetchin the new data based on the parameters previously done.

In [6]:
def fetch_weather_data(): # here i do the fetch of new data and drop old one to maintain real time
    try:
        response = requests.get(API_URL, params=PARAMS)
        if response.status_code != 200:
            print(f"Error: API request failed with status code {response.status_code}")
            return None

        data = response.json()  
        #print(json.dumps(data, indent=4))  
        
        if "hourly" not in data:
            raise KeyError("Missing 'hourly' key in API response")
        
        df = pd.DataFrame({
        "dt": pd.to_datetime(data["hourly"].get("time", [])),  
        "temp": data["hourly"].get("temperature_2m", [np.nan] * len(data["hourly"].get("time", []))),
        "pressure": data["hourly"].get("pressure_msl", [np.nan] * len(data["hourly"].get("time", []))),
        "humidity": data["hourly"].get("relative_humidity_2m", [np.nan] * len(data["hourly"].get("time", []))),
        "clouds": data["hourly"].get("cloudcover", [np.nan] * len(data["hourly"].get("time", []))),
        "wind_speed": data["hourly"].get("windspeed_10m", [np.nan] * len(data["hourly"].get("time", []))),
        "wind_deg": data["hourly"].get("winddirection_10m", [np.nan] * len(data["hourly"].get("time", []))),
        })
        df["season"] = df["dt"].dt.month.apply(get_season)
        return df

    except requests.exceptions.RequestException as e:
        print(f"Request error: {e}")
    except json.JSONDecodeError:
        print("Error decoding JSON response")
    except KeyError as e:
        print(f"KeyError: {e}")
    
    return None  


Here the function update teh dataset, so it concatenate the new data to the existing one , then drop the ancient records to ensure a real time dataset.

In [7]:

def update_dataset():
    new_data = fetch_weather_data()
    if new_data is None or new_data.empty:  
        print("No new data fetched. Skipping update.")
        return None  

    if os.path.exists(FILE_PATH):
        try:
            existing_data = pd.read_excel(FILE_PATH)
            df = pd.concat([existing_data, new_data]).drop_duplicates(subset=["dt"])
        except Exception as e:
            print(f"Error reading existing dataset: {e}")
            return None
    else:
        df = new_data

    df["dt"] = pd.to_datetime(df["dt"], errors="coerce")  

    df = df.sort_values("dt").iloc[-MAX_WEEKS * 7 * 24:] 

    try:
        df.to_excel(FILE_PATH, index=False)
        print(f"Dataset updated successfully. {len(df)} records saved.")
    except Exception as e:
        print(f"Error saving dataset: {e}")

    return df



Some Preprocessing and splitting for the data

In [8]:
from sklearn.preprocessing import StandardScaler

SEASON_MAPPING = {"Winter": 0, "Spring": 1, "Summer": 2, "Fall": 3}

def preprocess_data(df):
    df = df.copy()  
    df = df.dropna()  
    
    df["dt"] = pd.to_datetime(df["dt"])
    df["season"] = df["dt"].dt.month.apply(get_season)
    df["season"] = df["season"].map(SEASON_MAPPING) 
    
    X = df.drop(columns=["dt", "temp"]) 
    y = df["temp"]  
    
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
    
    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train)
    X_test_scaled = scaler.transform(X_test)
    
    return X_train_scaled, X_test_scaled, y_train, y_test, scaler  



Model training

In [9]:
def train_models(X_train, y_train):
    rf_model = RandomForestRegressor(n_estimators=100, random_state=42, n_jobs=-1)
    xgb_model = XGBRegressor(n_estimators=100, learning_rate=0.1, random_state=42, n_jobs=-1, verbosity=1)
    
    rf_model.fit(X_train, y_train)
    xgb_model.fit(X_train, y_train)
    
    return rf_model, xgb_model


Model evaluation based on test data

In [10]:
def evaluate_models(models, X_test, y_test):
    results = {}
    
    for name, model in models.items():
        y_pred = model.predict(X_test)
        
        results[name] = {
            "MAE": mean_absolute_error(y_test, y_pred),
            "MSE": mean_squared_error(y_test, y_pred),
             "R2": r2_score(y_test, y_pred)
           "RMSE": mean_squared_error(y_test, y_pred, squared=False),  
        }
    
    results_df = pd.DataFrame(results).T  
    #print(results_df)  
    
    return results_df


Implementation

In [14]:
if __name__ == "__main__":
    try:
        df = update_dataset()

        X_train, X_test, y_train, y_test, scaler = preprocess_data(df)

        rf_model, xgb_model = train_models(X_train, y_train)

        results = evaluate_models({"RandomForest": rf_model, "XGBoost": xgb_model}, X_test, y_test)

        print("\n Model Evaluation Metrics:")
        for model_name, metrics in results.iterrows():
            print(f"\n{model_name} Performance:")
            print(f"   - Mean Absolute Error (MAE): {metrics['MAE']:.4f}")
            print(f"   - Mean Squared Error (MSE): {metrics['MSE']:.4f}")
            print(f"   - Root Mean Squared Error (RMSE): {np.sqrt(metrics['MSE']):.4f}")  
            print(f"   - R² Score: {metrics['R2']:.4f}")

    except Exception as e:
        print(f"\n Error: {e}")
        
last_row= X_test[-1].reshape(1,-1)
predicted=  rf_model.predict(last_row)[0]
print(f"predicted temperature is:{predicted:.2f} degrees")


Dataset updated successfully. 45360 records saved.

 Model Evaluation Metrics:

RandomForest Performance:
   - Mean Absolute Error (MAE): 1.2884
   - Mean Squared Error (MSE): 3.0669
   - Root Mean Squared Error (RMSE): 1.7513
   - R² Score: 0.8788

XGBoost Performance:
   - Mean Absolute Error (MAE): 1.4394
   - Mean Squared Error (MSE): 3.5115
   - Root Mean Squared Error (RMSE): 1.8739
   - R² Score: 0.8613
predicted temperature is:23.76 degrees


C:\Users\ThinkBook\anaconda3\Lib\site-packages\sklearn\metrics\_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(
C:\Users\ThinkBook\anaconda3\Lib\site-packages\sklearn\metrics\_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(


### Conclusion
RandomForest is the better model for this dataset, providing more precise temperature predictions.<br>
Both models perform well (R² > 0.86), meaning they effectively capture relationships in the data.